In [1]:
import arcpy
from arcpy import env
import re
import pandas as pd
from arcpy.sa import *
import os

In [2]:
def raster_points(input_raster, output_raster, shp_file):
    # Read the input and check rasters
    in_raster = Raster(input_raster)
    ExtractMultiValuesToPoints(shp_file, [[in_raster, output_raster]], 'NONE')
    #arcpy.sa.ExtractValuesToPoints(thirty_points, in_raster, thirty_points, 'NONE')

In [3]:
# Define the parent folder and the geodatabase name
parent_folder = os.getcwd()
geodatabase_name = "example.gdb"

In [4]:
# Create the file geodatabase
geodatabase_path = os.path.join(parent_folder, geodatabase_name)
if os.path.exists(geodatabase_path):
    print(geodatabase_path)
else:
    arcpy.CreateFileGDB_management(parent_folder, geodatabase_name)

C:\Users\asanchez2415\Documents\example.gdb


In [5]:
# Set the geodatabase as the workspace
arcpy.env.workspace = geodatabase_path

In [6]:
# Set up the environment
arcpy.env.parallelProcessingFactor = 0
arcpy.CheckOutExtension("Spatial")
arcpy.env.matchMultidimensionalVariable = False
arcpy.env.overwriteOutput = True

In [7]:
# Set data product name
product = input("Enter the name of the data product: ")

Enter the name of the data product: MODIS


In [8]:
# Set the input directory
input_dir = input("Enter the input directory: ")

Enter the input directory: E:\share\MODIS_CONUS_NDVI\Bands


In [9]:
grid_shape_file = input("Enter the full file path and name of the grid shape file: ")

while not os.path.exists(grid_shape_file):
    print('The shapefile was not found')
    co = input("Please try again: ")

Enter the full file path and name of the grid shape file: E:\share\BIgRun\Grids\Grid_30.shp


In [29]:
temp_point_shape_path = input("Enter the full file path of the template point shape file: ")
temp_point_shape_name = input("Enter the name of the template point shape file: ")
temp_point_shape_file = os.path.join(temp_point_shape_path, temp_point_shape_name)
point_shape_name = input('Enter the name of the point shape file, you want to use: ')
point_check = input("Enter Y if you are adding to an existing point shape file, if you are creating/overwrite a new point file enter N")

print(point_shape_name)
# while not point_check != 'Y' and point_check != 'N':
#     point_shape_file = input("Please try again: ")


# if point_check == 'Y':
#     while not os.path.isfile(point_shape_file):
#         print('The shapefile was not found, should be in the same directory as the .py script')
#         point_shape_file = input("Please try again: ")

thirty_points = temp_point_shape_file
gb_thirty_points = os.path.join(geodatabase_path, point_shape_name)
print(gb_thirty_points)
if point_check == 'N':
    arcpy.conversion.FeatureClassToFeatureClass(thirty_points,geodatabase_path,point_shape_name)

Enter the full file path of the template point shape file: E:\share\BIgRun\Points
Enter the name of the template point shape file: Grid_30_Points.shp
Enter the name of the point shape file, you want to use: MODIS_Grid_30_Points2.shp
Enter Y if you are adding to an existing point shape file, if you are creating/overwrite a new point file enter NN
MODIS_Grid_30_Points2.shp
C:\Users\asanchez2415\Documents\example.gdb\MODIS_Grid_30_Points2.shp


In [34]:
# Read list of dates
text_file = open("dates.txt", "r")

while not os.path.exists("dates.txt"):
    print('The date list txt was not found, should be in the same directory as the .py script')
    n = input("Press enter to try again")
    text_file = open("dates.txt", "r")
date = text_file.read().split(',')
lag = input("Enter the number monthly lag: ")
lag = int(lag)
offset = input('Enter the day offset: ')
offset = int(offset)
start = input("If you need to start on a different date enter it's index, if not enter 1: ")
start = int(start) - 1
date = date[start:]

Enter the number monthly lag: 1
Enter the day offset: 4
If you need to start on a different date enter it's index, if not enter 1: 38


In [24]:
# File type
ftype = input("Enter file extention: ")
# Filename seperator
sep = input("Enter filename seperator: ")
#Date postiton
d_pos = input('Enter postion of the date with in the filenames, starting at index 0, example ALB_2000123 postion is 1:')
d_pos = int(d_pos)
# Year batch
yer = input("Enter the year you wish to process: to process all leave blank and press enter: ")
# Date Formating
d_f = input('If file use "%Y%j" type A, if "%Y%m%y" type B: ')
while d_f != 'A' and d_f != 'B':
    d = input('Input ERROR, If file use "%Y%j" type A, if "%Y%m%y" type B: ')
if d_f == 'A':
    d_format = '%Y%j'
if d_f == 'B':
    d_format = '%Y%m%d'

Enter file extention: .tif
Enter filename seperator: _
Enter postion of the date with in the filenames, starting at index 0, example ALB_2000123 postion is 1:10
Enter the year you wish to process: to process all leave blank and press enter: 
If file use "%Y%j" type A, if "%Y%m%y" type B: A


In [35]:
fdir = os.listdir(input_dir)
ref_dir = []
for fd in fdir:
    if fd.endswith(ftype):
        ref_dir.append(fd)
i = start
for d in date:
    j = False
    date_n = pd.to_datetime(d, format="%m/%d/%Y") + pd.DateOffset(months=lag)
    year = date_n.year
    u_date_n = date_n + pd.DateOffset(days=offset)
    l_date_n = date_n - pd.DateOffset(days=offset)
    for f in ref_dir:
        if j == True:
            break
        file_p = f.split(sep)
        file_p = re.sub("\D", "", file_p[d_pos])
        date_p = pd.to_datetime(file_p, format=d_format)
        if d_f == 'A':
            doy = date_n.timetuple().tm_yday
            u_date_c = str(u_date_n.year) + str(u_date_n.timetuple().tm_yday).zfill(3)
            l_date_c = str(l_date_n.year) + str(l_date_n.timetuple().tm_yday).zfill(3)
        if d_f == 'B':
            u_date_c = str(u_date_n.year)  + str(u_date_n.month).zfill(2) + str(u_date_n.day).zfill(2)
            l_date_c = str(l_date_n.year)  + str(l_date_n.month).zfill(2) + str(l_date_n.day).zfill(2)
        if date_p.year == year and (int(l_date_c) <= int(file_p) and int(file_p) <= int(u_date_c)):
            input_raster = os.path.join(input_dir, f)
            #Has to by date index number, because the arcpy multiopoint function has a name lenght limit
            data_name = product+"_"+ str(i).zfill(3)
            print(file_p)
            print(data_name + " " + str(date_n))
            output_raster = data_name
            raster_points(input_raster, output_raster, gb_thirty_points)
            j = True
    i = i + 1

2017164
MODIS_037 2017-06-15 00:00:00
2017171
MODIS_038 2017-06-22 00:00:00
2017178
MODIS_039 2017-06-29 00:00:00
2017185
MODIS_040 2017-07-05 00:00:00
2017192
MODIS_041 2017-07-12 00:00:00
2017199
MODIS_042 2017-07-19 00:00:00
2017206
MODIS_043 2017-07-26 00:00:00
2017213
MODIS_044 2017-08-03 00:00:00
2017220
MODIS_045 2017-08-10 00:00:00
2017227
MODIS_046 2017-08-17 00:00:00
2017234
MODIS_047 2017-08-24 00:00:00
2017241
MODIS_048 2017-08-31 00:00:00
2017248
MODIS_049 2017-09-07 00:00:00
2017255
MODIS_050 2017-09-14 00:00:00
2017262
MODIS_051 2017-09-21 00:00:00
2017269
MODIS_052 2017-09-28 00:00:00
2017276
MODIS_053 2017-10-04 00:00:00
2017283
MODIS_054 2017-10-11 00:00:00
2017290
MODIS_055 2017-10-18 00:00:00
2017297
MODIS_056 2017-10-25 00:00:00
2017304
MODIS_057 2017-11-02 00:00:00
2017311
MODIS_058 2017-11-09 00:00:00
2017318
MODIS_059 2017-11-16 00:00:00
2017325
MODIS_060 2017-11-23 00:00:00
2017332
MODIS_061 2017-11-30 00:00:00
2018121
MODIS_062 2018-05-02 00:00:00
2018128
MODI

In [46]:
arcpy.ValidateTableName(gb_thirty_points)

'C__Users_asanchez2415_Documents_example_gdb_LAI_Grid_30_Points_shp'

In [109]:
len(os.listdir(input_dir))

100

In [21]:
print(ftype)

.img
